In [1]:
import os
import socket
import gymnasium as gym
import json
import time
import subprocess
import struct, sys, time
import uuid

from stable_baselines3 import PPO

from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.evaluation import evaluate_policy

In [ ]:
# class MonopolyDealClient:
#     def __init__(self, host='localhost', port=27599):
#         self.host = host
#         self.port = port
#         self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
#         self.sock.connect((host, port))
#         print("Connected to Monopoly Deal server!")

#     def send(self, message):
#         self.sock.sendall((message + "\n").encode())

#     def receive(self):
#         return self.sock.recv(4096).decode()

#     def close(self):
#         self.sock.close()


In [ ]:
# class MonopolyDealEnv(gym.Env):
#     def __init__(self, host='localhost', port=12345):  # Adjust port based on server
#         super().__init__()
#         # Start the server (optional, if not running in another terminal)
#         self.server_process = subprocess.Popen(['mvn', 'exec:java', '-pl', 'server'], cwd='server')
#         # Connect to the server
#         self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
#         self.sock.connect((host, port))
#         # Define action and observation spaces (customize based on game)
#         self.action_space = gym.spaces.Discrete(10)  # Example: 10 possible actions (e.g., play card 0-9)
#         self.observation_space = gym.spaces.Dict({  # Example: game state as a dictionary
#             "board": gym.spaces.Box(low=0, high=100, shape=(10,), dtype=int),
#             "hand": gym.spaces.Box(low=0, high=100, shape=(5,), dtype=int)
#         })

#     def reset(self, seed=None, options=None):
#         # Send reset command (adjust based on server protocol)
#         self.sock.sendall("RESET\n".encode())  # Add \n if server expects it
#         response = self.sock.recv(1024).decode()
#         try:
#             state = json.loads(response)  # Assume JSON response
#         except json.JSONDecodeError:
#             state = response  # Fallback to raw text if not JSON
#         return state, {}

#     def step(self, action):
#         # Send action (adjust format based on server)
#         self.sock.sendall(f"PLAY_CARD {action}\n".encode())
#         response = self.sock.recv(1024).decode()
#         try:
#             data = json.loads(response)
#             state = data.get("state", {})
#             reward = data.get("reward", 0.0)
#             done = data.get("done", False)
#             info = data.get("info", {})
#         except json.JSONDecodeError:
#             # Fallback if response is not JSON
#             state = response
#             reward = 0.0
#             done = False
#             info = {}
#         return state, reward, done, False, info

#     def close(self):
#         self.sock.close()
#         if hasattr(self, 'server_process'):
#             self.server_process.terminate()

In [ ]:
# class MonopolyDealEnv(gym.Env):
#     def __init__(self, host='localhost', port=27599):
#         super().__init__()
#         self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
#         self.sock.connect((host, port))
#         self.sock.sendall("addbot RLBot\n".encode())
#         self.action_space = gym.spaces.Discrete(20)
#         self.observation_space = gym.spaces.Dict({
#             "hand": gym.spaces.Box(low=0, high=100, shape=(5,), dtype=int),
#             "board": gym.spaces.Box(low=0, high=100, shape=(10,), dtype=int),
#             "turn": gym.spaces.Discrete(4)
#         })

#     def reset(self, seed=None, options=None):
#         self.sock.sendall("reset\n".encode())
#         time.sleep(0.1)
#         response = self.sock.recv(4096).decode()
#         try:
#             state = json.loads(response)
#         except json.JSONDecodeError:
#             state = {"raw": response}
#         self.sock.sendall("start\n".encode())
#         time.sleep(0.1)
#         response = self.sock.recv(4096).decode()
#         try:
#             state = json.loads(response)
#         except json.JSONDecodeError:
#             state = {"raw": response}
#         return state, {}

#     def step(self, action):
#         commands = {0: "nextturn", 1: "createcard action dealbreaker"}
#         command = commands.get(action, f"PLAY_CARD {action}")
#         self.sock.sendall(f"{command}\n".encode())
#         time.sleep(0.1)
#         response = self.sock.recv(4096).decode()
#         try:
#             data = json.loads(response)
#             state = data.get("state", {})
#             reward = data.get("reward", 0.0)
#             done = data.get("done", False)
#             info = data.get("info", {})
#         except json.JSONDecodeError:
#             state = {"raw": response}
#             reward = 0.0
#             done = False
#             info = {"error": "Non-JSON response"}
#         return state, reward, done, False, info

#     def close(self):
#         self.sock.sendall("stop\n".encode())
#         self.sock.close()




In [ ]:
# Simple example to test the connection works
# client = MonopolyDealClient()
# client.send("HELLO")
# print(client.receive())
# client.close()

In [2]:
HOST = "127.0.0.1"
PORT = 27599

In [ ]:
# Connect to server
# sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# sock.connect(('localhost', 27599))
# print("Connected to server")

In [10]:
def write_utf(s):
    encoded = s.encode("utf-8")
    length = len(encoded)
    return length.to_bytes(2, byteorder="big") + encoded


In [11]:
# PacketInitiateLogin ID = 2 (registered order in NetHandler), protocolVersion = 24
PACKET_ID = 2
PROTOCOL_VERSION = 24

sock = socket.create_connection((HOST, PORT))

# build payload: single int (protocolVersion)
payload = struct.pack(">i", PROTOCOL_VERSION)
payload_len = len(payload)

# Header as separate fields, not combined struct
header = struct.pack(">h", PACKET_ID) + struct.pack(">i", payload_len)
msg = header + payload

sock.sendall(msg)
print("Sent PacketInitiateLogin(protocolVersion=24)")

# try to read a response header (2 bytes id + 4 bytes len)
hdr = sock.recv(6)
if len(hdr) < 6:
    print("No response header received (got bytes):", hdr)
else:
    pkt_id, pkt_len = struct.unpack(">hI", hdr)
    print("Received packet id:", pkt_id, "payload length:", pkt_len)
    if pkt_len:
        data = b''
        while len(data) < pkt_len:
            chunk = sock.recv(pkt_len - len(data))
            if not chunk:
                break
            data += chunk
        print("Payload bytes:", data)

# keep socket open


Sent PacketInitiateLogin(protocolVersion=24)
Received packet id: 4 payload length: 24
Payload bytes: b'\x00\x00\x00\x18\x00\x00\x00\x10n/\xd5\x0f\x9e\xb6\xb1\xe7X5l\x1a\x02$\xbc\xab'


In [ ]:
# # payload is whatever you just received for packet id 4
# protocol_version = int.from_bytes(payload[0:4], "big")
# # everything after the first 4 bytes
# server_key_bytes = payload[4:]

# print("Protocol:", protocol_version)
# print("Server key bytes:", server_key_bytes)

In [9]:
# PacketLogin ID = 3
PACKET_ID = 3
CLIENT_VERSION = "1.0.0"
PLAYER_NAME = "Logan"
UUID_BYTES = uuid.uuid4().bytes

# 1. protocolVersion (int)
proto_bytes = struct.pack(">i", PROTOCOL_VERSION)

# 2. clientVersion (String) - FIXED: int length prefix + UTF-16BE chars
client_version_chars = CLIENT_VERSION.encode('utf-16be')
client_version_payload = struct.pack(">i", len(CLIENT_VERSION)) + client_version_chars  # int length, not short

# 3. id (byte[] - 16 bytes)
id_payload = struct.pack(">i", 16) + UUID_BYTES  # Length prefix (16) + 16 bytes of UUID

# 4. name (String) - FIXED: int length prefix + UTF-16BE chars  
name_chars = PLAYER_NAME.encode('utf-16be')
name_payload = struct.pack(">i", len(PLAYER_NAME)) + name_chars  # int length, not short

# Combine all fields
payload = proto_bytes + client_version_payload + id_payload + name_payload

# Header: packet ID (short) + payload length (int)
header = struct.pack(">h", PACKET_ID) + struct.pack(">i", len(payload))
msg = header + payload

print(f"Payload length: {len(payload)}")
print(f"Client version payload: {client_version_payload.hex()}")
print(f"Name payload: {name_payload.hex()}")

# Send the login packet
sock.sendall(msg)
print("Sent PacketLogin with corrected string format!")

# Read response
hdr = sock.recv(6)
if len(hdr) == 6:
    pkt_id = struct.unpack(">h", hdr[:2])[0]
    pkt_len = struct.unpack(">i", hdr[2:6])[0]
    print("Received packet id:", pkt_id, "payload length:", pkt_len)
    
    if pkt_len > 0:
        data = b''
        while len(data) < pkt_len:
            chunk = sock.recv(pkt_len - len(data))
            if not chunk:
                break
            data += chunk
        print("Login response payload:", data.hex())
    else:
        print("Login successful (no payload)")

Payload length: 52
Client version payload: 000000050031002e0030002e0030
Name payload: 00000005004c006f00670061006e
Sent PacketLogin with corrected string format!
Received packet id: 39 payload length: 8
Login response payload: 0000000100000000


In [ ]:
# Send over socket
sock.sendall(msg)
print("Sent!")

# Optionally read server response
hdr = sock.recv(6)
if len(hdr) == 6:
    pkt_id, pkt_len = struct.unpack(">hI", hdr)
    print("Received packet id:", pkt_id, "payload length:", pkt_len)
    data = b''
    while len(data) < pkt_len:
        chunk = sock.recv(pkt_len - len(data))
        if not chunk:
            break
        data += chunk
    print("Payload bytes:", data)


In [ ]:
# 2) Build PacketLogin
# Packet ID for PacketLogin comes from NetHandler order.
# Looking at NetHandler.java, PacketInitiateLogin is 2, PacketLogin is 3.
PACKET_LOGIN_ID = 3

player_name = "RLAgent"
player_name_bytes = player_name.encode("utf-8")

# format = [string length (int32)] + [string bytes] + [raw key bytes]
login_payload = struct.pack(">I", len(player_name_bytes)) + player_name_bytes + server_key

# Prepend packet ID (short) and payload length (int)
packet = struct.pack(">hI", PACKET_LOGIN_ID, len(login_payload)) + login_payload

In [ ]:
print("Player name length:", len(player_name_bytes))
print("Server key length:", len(server_key))
print("Total payload length:", len(login_payload))

In [ ]:
# Example usage:
player_name = "example_player_name"
server_key_bytes = b'n/\xd5\x0f\x9e\xb6\xb1\xe7X5l\x1a\x02$\xbc\xab'

payload = write_utf(player_name) + server_key_bytes  # server_key_bytes is the 16-byte key you received
print(payload)

In [ ]:
packet_id = 3  # adjust to actual registered ID
packet_bytes = packet_id.to_bytes(2, byteorder="big") + len(payload).to_bytes(4, byteorder="big") + payload

In [ ]:
# 3) Send it to the server
sock.sendall(packet)
print("Sent PacketLogin with name", player_name)

In [ ]:
# # Send a simple command
# sock.sendall("reset\n".encode())
# time.sleep(0.5)  # Wait for response
# response = sock.recv(4096).decode()
# print("Response:", response)

In [ ]:
# sock.sendall(b"addbot TestBot\n")  # Try adding a bot
# time.sleep(0.5)
# try:
#     response = sock.recv(1024)
#     print("Raw response (bytes):", response)
#     print("Decoded response:", response.decode('utf-8', errors='replace'))
# except socket.timeout:
#     print("No response received within 5 seconds")
# sock.close()

In [ ]:
# # Send a minimal message to mimic client handshake
# sock.sendall(b"HELLO\n")  # Placeholder, adjust if protocol is known
# time.sleep(0.5)

In [ ]:
# try:
#     response = sock.recv(1024)
#     print("Raw response (bytes):", response)
#     print("Decoded response:", response.decode('utf-8', errors='replace'))
#     # Try addbot after potential handshake
#     sock.sendall(b"addbot TestBot\n")
#     time.sleep(0.5)
#     response = sock.recv(1024)
#     print("Addbot response (bytes):", response)
#     print("Addbot decoded:", response.decode('utf-8', errors='replace'))
# except socket.timeout:
#     print("No response received within 5 seconds")
# sock.close()

In [ ]:
# # Try another command
# sock.sendall("start\n".encode())
# time.sleep(0.5)
# response = sock.recv(4096).decode()
# print("Response:", response)

In [ ]:
# sock.close()

In [ ]:
env = MonopolyDealEnv()
state, info = env.reset()
print("Initial state:", state)
next_state, reward, done, truncated, info = env.step(0)  # Example: nextturn
print("Next state:", next_state, "Reward:", reward, "Done:", done, "Info:", info)
env.close()

In [ ]:
client = MonopolyDealClient()

# Start the game
# deals 5 cards to all players
client.send("/start")
print(client.receive())

In [ ]:
client.send("/addbot Bot1")
print(client.receive())

client.send("/addbot Bot2")
print(client.receive())

In [ ]:
def start_game(self):
    self.send("/start")
    return self.receive()

def add_bot(self, name):
    self.send(f"/addbot {name}")
    return self.receive()
